In [1]:
from pt.evaluation.pascal_voc_evaluation_copy import PascalVOCDetectionEvaluator
#from detectron2.evaluation import PascalVOCDetectionEvaluator
from detectron2.evaluation import inference_on_dataset

In [2]:
from Nb_utils import *

/home/superorange5/.local/lib/python3.7/site-packages/torchvision/transforms/transforms.py:892: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  "Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. "


## setting (model path & config path)

In [3]:
DA_dataset = 'c2kb' #'c2kb'  #skf2c
detector_name = 'ptso' #'ptda'  #'ptso'

In [4]:
if DA_dataset == 'c2kb':
    ratio=1.2
    val_dataset_name = 'VOC2007_bddval1'   #'VOC2007_bddvalsmall'
    source_list =['kitti','cityscape']
    config_path = "configs/pt/final_c2b.yaml"
    if detector_name == 'ptda':
        model_c2b = load_TSmodel('c2b', 11999)
        model_k2b = load_TSmodel('k2b', 'final')
        model_list=[model_k2b,model_c2b]
    elif detector_name == 'ptso':
        model_c2b = load_TSmodel('c2b_source-only', 27999)
        model_k2b = load_TSmodel('k2b_source-only', 7999)
        model_list=[model_k2b,model_c2b]

elif DA_dataset=='skf2c':    
    val_dataset_name = 'VOC2007_cityval1'
    source_list =['sim10k','kitti','foggycityscape']
    ratio = 1.706
    config_path = "configs/pt/final_k2c.yaml"
    if detector_name == 'ptda':
        model_s2c = load_TSmodel('s2c', 15999)
        model_k2c = load_TSmodel('k2c', 19999)
        #model_f2c = get_model('f2c', 'final') #8class
        model_f2c = load_TSmodel('f2c_1class', 11999)
        model_list=[model_s2c,model_k2c,model_f2c]
    elif detector_name == 'ptso':
        model_s2c = load_TSmodel('s2c_source-only', 23999)
        model_k2c = load_TSmodel('k2c_source-only', 'final')
        model_f2c = load_TSmodel('f2c_source-only', 27999)
        model_list=[model_s2c,model_k2c,model_f2c]

        
# config
cfg = setup_all(config_path)

# load test data
test_data_loader = build_detection_test_loader(cfg, val_dataset_name)
evaluator = PascalVOCDetectionEvaluator(val_dataset_name)

keep_experiments/c2b_source-only/cfg.yaml
keep_experiments/c2b_source-only/model_0027999.pth


AssertionError: Config file 'keep_experiments/c2b_source-only/cfg.yaml' does not exist!

# evaluation

### A. inference all

In [ ]:
results_i = inference_on_dataset(model_to_eval, test_data_loader, evaluator)


In [19]:
results_i

OrderedDict([('bbox',
              {'AP': 29.757639038918928,
               'AP50': 58.169071629659264,
               'AP75': 29.25375686697318})])

### B. inference batch

In [7]:
from contextlib import ExitStack, contextmanager
from torch import nn
 
evaluator.reset()
model_to_eval = model_list[0]
model_to_eval.eval()
#print(instances.pred_boxes.tensor)

for idx, inputs in enumerate(test_data_loader):
    #print(inputs)
    #print(get_proposal_roih(inputs,model_to_eval))
    outputs = model_to_eval.modelStudent(inputs)
    #print(outputs[0]["instances"])
    if torch.cuda.is_available():
        torch.cuda.synchronize()

    evaluator.process_eval(inputs, outputs)
results = evaluator.evaluate()

In [8]:
results

OrderedDict([('bbox',
              {'AP': 29.507701847189956,
               'AP50': 52.372955971225146,
               'AP75': 29.56818902766315})])

In [9]:
outputs

[{'instances': Instances(num_instances=28, image_height=1024, image_width=2048, fields=[pred_boxes: Boxes(tensor([[ 326.7519,  440.0365,  618.3756,  519.4741],
          [   0.0000,  404.9550,  168.9063,  550.9739],
          [1182.1501,  444.9689, 1252.7135,  496.8851],
          [ 134.5826,  405.4463,  355.9171,  536.6377],
          [1858.5892,   63.9379, 1954.0305,  177.9242],
          [ 504.0312,  439.1638,  609.7053,  524.9960],
          [ 905.1487,  442.3421, 1010.9626,  499.1852],
          [   6.4120,  398.8298,  406.4436,  547.3654],
          [1221.2854,  511.4424, 1292.7891,  555.7824],
          [ 927.2269,  456.6595,  988.6171,  497.8509],
          [1481.2874,  184.6058, 1748.6989,  292.9760],
          [1484.6484,  184.5870, 1621.5295,  273.5999],
          [ 953.2852,  452.2430, 1005.0145,  493.9701],
          [ 978.5691,  437.3337, 1024.8195,  472.8095],
          [ 514.1945,  485.8729,  576.8962,  524.5416],
          [1152.2059,  422.3804, 1200.3099,  465.1030],


## pseudo-label generation

In [5]:
    def get_match_array_nogt(proposals_roih):
        source_num = len(proposals_roih)
        batch_size = len(proposals_roih[0])

        batch_match_array= []
        for data_idx in range(batch_size):
            match_array_source = []
            for i, source_prediction_n in enumerate(proposals_roih):
                match_array_source_n =[]
                # others
                #pairwise_sa_sb = []        
                for j in range(source_num):
                    if j!=i:
                        sourcen_n_match_other = structures.pairwise_iou(source_prediction_n[data_idx].get('pred_boxes'),proposals_roih[j][data_idx].get('pred_boxes'))
                        #pairwise_sa_sb.append(sourcen_n_match_other)
                        match_array_source_n.append(get_match_array(sourcen_n_match_other))
                match_array_source.append(match_array_source_n)
            batch_match_array.append(match_array_source)

        return  batch_match_array

In [6]:
    def bb_ensemble(mt_src,src_idx):
        source_num = len(mt_src[0])
        df_src = pd.DataFrame()    
        src_array = np.array(mt_src[src_idx]).T
        df_src = pd.DataFrame(src_array)
        df_src['summary'] = df_src.sum(axis=1)
        keep_index = df_src.index[df_src.summary==source_num]
        return keep_index

In [7]:
    def process_pseudo_label2(proposals_roih):
        list_instances = []
        for proposal_bbox_inst in proposals_roih:

            image_shape = proposal_bbox_inst.image_size
            new_proposal_inst = FreeInstances(image_shape)

            new_bbox_loc = proposal_bbox_inst.pred_boxes.tensor
            pseudo_boxes = Boxes(new_bbox_loc)

            # add boxes to instances
            
            new_proposal_inst.pseudo_boxes = pseudo_boxes
            new_proposal_inst.scores_logists = proposal_bbox_inst.scores_logists
            new_proposal_inst.scores = proposal_bbox_inst.scores
            new_proposal_inst.pred_classes = proposal_bbox_inst.pred_classes

            
            if proposal_bbox_inst.has('boxes_sigma'):
                new_proposal_inst.boxes_sigma = proposal_bbox_inst.boxes_sigma
            list_instances.append(new_proposal_inst)
        return list_instances

In [8]:
def scale_return_proposal(proposal_roih,ratio):
    scale_pred = proposal_roih[0].get('pred_boxes')
    scale_pred.scale(ratio,ratio)
    proposal_roih[0].pred_boxes=scale_pred
    return proposal_roih

In [25]:
_SRC_IDX = 1#  0:s  1:k 2:foggy, 0:k 1:c


In [26]:

from pt.structures.instances import FreeInstances

#data_loader_iter = iter(test_data_loader)
#test_data = data_loader_iter.next()


# change to training mode
for model in model_list:
    model.train()
    
    
evaluator.reset()

for idx, test_data in enumerate(test_data_loader):
    #print(idx)

    with torch.no_grad():
        roih_list = []
        for model in model_list:
            ( _, proposals_rpn_unsup_k, proposals_roih_unsup_k, _,) = model.modelStudent(test_data, branch="unsup_data_weak")
            roih_list.append(proposals_roih_unsup_k)
            
        batch_size = len(roih_list[0])
        mt_src = get_match_array_nogt(roih_list)



        pesudo_proposals_roih_combined = []
        for batch_idx in range(batch_size):
            keep_index = bb_ensemble(mt_src[batch_idx],_SRC_IDX)
            
            pesudo_proposals_roih_combined.append(roih_list[_SRC_IDX][batch_idx][keep_index])
        pesudo_proposals_roih_combined_scaled = scale_return_proposal(pesudo_proposals_roih_combined,ratio)
        pesudo_proposals_roih_combined_final = process_pseudo_label2(pesudo_proposals_roih_combined_scaled)
        
        evaluator.process_train(test_data, pesudo_proposals_roih_combined_final)
results = evaluator.evaluate()

In [27]:
results

OrderedDict([('bbox',
              {'AP': 31.479858580220387,
               'AP50': 52.231894248329205,
               'AP75': 32.95418627309308})])

In [30]:
proposals_roih_unsup_k[0].get('pseudo_boxes')

In [15]:
instances = pesudo_proposals_roih_combined_final[0].to(torch.device("cpu"))
 

In [16]:
boxes = instances.pseudo_boxes.tensor.numpy()

In [18]:
scores = instances.scores.tolist()

In [19]:
classes = instances.pred_classes.tolist()

In [32]:
instances.pseudo_boxes

Boxes(tensor([[2.0703e+02, 2.5250e+02, 3.5305e+02, 3.0611e+02],
        [6.9360e+02, 2.6129e+02, 7.3344e+02, 2.9227e+02],
        [2.3941e+01, 2.2972e+02, 2.0330e+02, 3.1684e+02],
        [5.6760e-01, 2.3787e+02, 9.7256e+01, 3.2250e+02],
        [5.2475e+02, 2.5986e+02, 5.9003e+02, 2.8800e+02],
        [3.0422e+02, 2.5544e+02, 3.5472e+02, 3.0591e+02],
        [6.7413e+02, 2.5055e+02, 7.0496e+02, 2.7166e+02],
        [5.1500e+02, 2.6150e+02, 5.5781e+02, 2.8730e+02],
        [6.7304e+02, 2.3496e+02, 7.0752e+02, 2.6571e+02],
        [6.7183e+02, 2.5848e+02, 6.9716e+02, 2.7603e+02],
        [6.6311e+02, 2.5212e+02, 6.9341e+02, 2.7043e+02],
        [7.1064e+02, 2.4688e+02, 7.4932e+02, 2.7358e+02],
        [6.5555e+02, 2.5004e+02, 6.8373e+02, 2.6945e+02]]))

In [24]:
for box, score, cls in zip(boxes, scores, classes):
    xmin, ymin, xmax, ymax = box
    # The inverse of data loading logic in `datasets/pascal_voc.py`
    xmin += 1
    ymin += 1
    print(f" {score:.3f} {xmin:.1f} {ymin:.1f} {xmax:.1f} {ymax:.1f}")
    

 0.975 208.0 253.5 353.0 306.1
 0.968 694.6 262.3 733.4 292.3
 0.951 24.9 230.7 203.3 316.8
 0.936 1.6 238.9 97.3 322.5
 0.890 525.7 260.9 590.0 288.0
 0.124 305.2 256.4 354.7 305.9
 0.040 675.1 251.5 705.0 271.7
 0.016 516.0 262.5 557.8 287.3
 0.015 674.0 236.0 707.5 265.7
 0.013 672.8 259.5 697.2 276.0
 0.012 664.1 253.1 693.4 270.4
 0.009 711.6 247.9 749.3 273.6
 0.006 656.5 251.0 683.7 269.4


# draw

In [28]:
cfg = setup("configs/pt/final_k2c.yaml")
dataset_name = 'VOC2007_cityval1'
test_data_loader = build_detection_test_loader(cfg, dataset_name)
for test_data in test_data_loader:
    target_metadata = MetadataCatalog.get(dataset_name)

In [29]:
target_metadata

namespace(name='VOC2007_cityval1',
          thing_classes=['car'],
          dirname='data/VOC2007_cityval',
          year=2012,
          split='val',
          evaluator_type='pascal_voc')

In [33]:
bboxes_to_draw = instances.pseudo_boxes

In [27]:
pesudo_proposals_roih_combined_final[0].to(torch.device("cpu"))

FreeInstances(num_instances=0, image_height=600, image_width=1200, fields=[pseudo_boxes: Boxes(tensor([], size=(0, 4))), scores_logists: tensor([], size=(0, 2)), scores: tensor([]), pred_classes: tensor([], dtype=torch.int64), boxes_sigma: tensor([], size=(0, 4))])

In [16]:
data_loader_iter = iter(test_data_loader)
test_data = data_loader_iter.next()

In [17]:
model_to_eval = model_list[0]



* eval

In [38]:
model_to_eval.eval()
outputs = model_to_eval.modelStudent(test_data)

In [39]:
outputs

[{'instances': Instances(num_instances=48, image_height=1024, image_width=2048, fields=[pred_boxes: Boxes(tensor([[ 146.5441,  385.5511,  448.0932,  536.4307],
          [1564.1633,  259.5304, 2048.0000,  777.3774],
          [1327.8706,  367.4927, 1659.8536,  568.8884],
          [1123.5928,  396.8085, 1204.7588,  453.3437],
          [ 836.7784,  392.7728,  934.2477,  448.4558],
          [   0.0000,  362.3156,   79.4561,  587.6072],
          [ 696.6040,  368.8929,  784.0499,  442.3494],
          [1329.2783,  380.4111, 1444.3099,  543.8692],
          [1087.3569,  396.1790, 1141.2621,  445.6662],
          [ 763.1310,  375.6651,  825.3677,  435.2816],
          [ 120.4234,  343.5369,  211.7279,  433.3838],
          [1048.1008,  367.9299, 1106.7754,  426.8610],
          [1586.8948,  369.6127, 1912.4065,  733.5382],
          [1360.7120,  247.0661, 1407.4061,  296.7185],
          [ 106.8375,  313.8461,  212.4159,  482.0017],
          [ 971.8511,  392.9136, 1041.5619,  426.9579],


* train

In [18]:
model_to_eval.train()
( _, proposals_rpn_unsup_k, proposals_roih_unsup_k, _,) = model_to_eval.modelStudent(test_data, branch="unsup_data_weak")